<a href="https://colab.research.google.com/github/ivanovjaroslaw/hometasks/blob/main/ML_PT_Course/ML_PTCourse_MicroHT1_alternative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Микро-ДЗ 1: Знакомство с машинным обучением
## Альтернативный трек: Прогноз стоимости алмазов с воспроизведённым с нуля пайплайном

**Подготавливаем** окружение и загружаем данные

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

RANDOM_STATE = 42
df = pd.read_csv("https://raw.githubusercontent.com/aiedu-courses/stepik_eda_and_dev_tools/main/datasets/diamonds_good.csv")

Взглянем на данные

In [5]:
df.head(n = 10)

,carat,cut,color,clarity,depth,table,price,'x','y','z'
0,0.23,Ideal,E,SI2,61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335.0,4.34,4.35,2.75
5,NaN,Very Good,J,VVS2,62.8,57.0,336.0,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336.0,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337.0,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337.0,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338.0,4.00,4.05,2.39


In [6]:
df.shape

(53940, 10)

Попробуем **проинтерпретировать данные** (через быстрый гуглёж):
* `carat` - караты в каких-то единицах, возможно важный "признак"
* `cut` - качество огранки
* `color` - похоже на общую прозрачность/цветастость --- шкалу "GIA" (от D до Z), где чем дальше, тем более мутное
* `clarity` - тоже про "прозрачность", но не общую, а про "включения" -- сложная величина; в порядке убывания ценности (FL/IF, VVS1/VVS2, VS1/VS2, SI1/SI2, I1, I2, I3)
* `depth` - высота алмаза? с виду процентная, видимо, как отношение общей глубины (высоты?) к минимальному диаметру
* `table` - процент какой-то там площадки верхней грани, видимо, к общему "диаметру" алмаза в ширину
* `price` - ага, наша целевая переменная, предположим что в $, потому что маленькие караты
* `x`, `y`, `z` - скорее всего трёхмерный размер короба, в который вписывается алмаз, возможно в каких-нибудь американских дюймах


Далее **оценим качество** данных перед тренировкой (с виду нужная глубока подготовка, но я не знаю как и нужного много времени и знания API):
* Явно есть `NaN` в `carat` да и возможно в других колонках. Не ясно как это правильно перевести в числа (точно не ноль) или как-то иначе учесть. Предположим, что `NaN` сработает и другого вида пропусков нет.
* Колонки `cut`, `color`, `clarity` - категориальные признаки и нужно кодирование. Однако `one-hot` здесь явно не подходит, потому что есть заданное направление ценности между категориями
* Нужно проверить и обработать потенциальные выбросы
* Явно не зря в первой домашке смотрели на корреляцию между признаками, но дальше просмотра это не двинулось. Зачем смотрели корреляцию - что это может дать и что мы можем дополнительно подшаманить из-за этого?




Продолжим **подготовку** данных

In [8]:
X = df.drop(["price", "cut", "color", "clarity"], axis=1);
y = df["price"]

Сделаем **разбиение** на обучающую и тестовую выборки

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, X_test.shape

((40455, 6), (13485, 6))

Далее **обучаем линейную модель**

In [10]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# (?) а вот и сработал "NaN" - ошибка - надо его как-то обработать
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values